<h1 style="font-family:Times New Roman;">Kaggle Competition</h1>

<p style="font-family:Times New Roman;"><b><font size = "4">Suhas Alur</font></b></p>

<h1 style="font-family:Times New Roman;">Loading & Pre-processing</h1>

In [1]:
# import necessary libraries and specify that graphs should be plotted inline
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm, datasets
%cd 'C:\Users\Suhas\Documents\MSBA\Semester Two\Predictive Analytics\Homework\Homework - 5\Data'

C:\Users\Suhas\Documents\MSBA\Semester Two\Predictive Analytics\Homework\Homework - 5\Data


In [40]:
# Import the train dataset
data_train = pd.read_csv('train.csv')

In [41]:
# Splitting the dataset based on attributes and target variable
X = data_train.iloc[:,0:25]
y = data_train.adopter

In [42]:
X = np.array(X)
y = np.array(y)

In [43]:
# Splitting the train data into training and testing

from sklearn.model_selection import train_test_split
X, X_submission, y, y_suhas = train_test_split(X, y, test_size=0.3, random_state=42)

In [44]:
# Import the train dataset
#data_test = pd.read_csv('test.csv')

In [45]:
# Taking only the required attributes
#X_submission = data_test.iloc[:,0:25]
#X_submission = np.array(X_submission)

In [46]:
from __future__ import division
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [47]:
def logloss(attempt, actual, epsilon=1.0e-15):
    """Logloss, i.e. the score of the bioresponse competition.
    """
    attempt = np.clip(attempt, epsilon, 1.0-epsilon)
    return - np.mean(actual * np.log(attempt) +
                     (1.0 - actual) * np.log(1.0 - attempt))

In [48]:
np.random.seed(0)  # seed to shuffle the train set
n_folds = 10
verbose = True
shuffle = False

In [49]:
if shuffle:
    idx = np.random.permutation(y.size)
    X = X[idx]
    y = y[idx]

In [50]:
skf = list(StratifiedKFold(y, n_folds))

In [51]:
clfs = [RandomForestClassifier(n_estimators = 100, max_depth = 10, max_features = 7, min_samples_split = 4, bootstrap = True, oob_score = True, n_jobs = -1, class_weight = 'balanced', random_state = 42, criterion='gini'),
        RandomForestClassifier(n_estimators = 100, max_depth = 10, max_features = 7, min_samples_split = 4, bootstrap = True, oob_score = True, n_jobs = -1, class_weight = 'balanced', random_state = 42, criterion='entropy'),
        GradientBoostingClassifier(loss = 'deviance', learning_rate=0.05, min_weight_fraction_leaf = 0.1, min_samples_split=2, n_estimators=80, max_depth=20, subsample=0.1,random_state=42),
        GradientBoostingClassifier(loss = 'exponential', learning_rate=0.05, min_weight_fraction_leaf = 0.1, min_samples_split=2, n_estimators=80, max_depth=20, subsample=0.1,random_state=42),
        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='gini',random_state=42),
        ExtraTreesClassifier(n_estimators=100, n_jobs=-1, criterion='entropy',random_state=42),
        xgb.XGBClassifier(learning_rate =0.8, n_estimators=100, max_depth=2, min_child_weight=6, gamma=5, subsample=1, colsample_bytree=0.5, reg_alpha=0.1, objective= 'binary:logistic', nthread=10, scale_pos_weight=1, seed=42),
        xgb.XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=2, min_child_weight=6, gamma=5, subsample=1, colsample_bytree=0.1, reg_alpha=0.01, objective= 'binary:logistic', nthread=20, scale_pos_weight=1, seed=42)]

In [52]:
print "Creating train and test sets for blending."

dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_submission.shape[0], len(clfs)))

for j, clf in enumerate(clfs):
    print j, clf
    dataset_blend_test_j = np.zeros((X_submission.shape[0], len(skf)))
    for i, (train, test) in enumerate(skf):
        print "Fold", i
        X_train = X[train]
        y_train = y[train]
        X_test = X[test]
        y_test = y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_submission)[:, 1]
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)

Creating train and test sets for blending.
0 RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=10, max_features=7,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
1 RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='entropy', max_depth=10, max_features=7,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=4,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
            oob_score=True, random_state=42, verbose=0, warm_start=False)
Fold 0
Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8
Fold 9
2 GradientBoostingClassifier(criterion='friedman

In [53]:
print
print "Blending..."
clf = LogisticRegression(random_state = 9, C = 0.3, n_jobs = -1, penalty = 'l1', class_weight = 'balanced', intercept_scaling  = 1, multi_class = 'ovr', fit_intercept = False)
clf.fit(dataset_blend_train, y)
y_submission = clf.predict_proba(dataset_blend_test)[:, 1]

# 
#print "Linear stretch of predictions to [0,1]"
#y_submission = (y_submission - y_submission.min()) / (y_submission.max() - y_submission.min())


Blending...


In [54]:
y_submission[1]

0.47909544421871447

In [55]:
from sklearn.metrics import classification_report, confusion_matrix

def drange(start, stop, step):
    while start < stop:
            yield start
            start += step


for j in drange(0.0, 1.0, 0.01):
    cutoff = j
    y_pred = list()
    for i in range(0,len(y_submission)):
        if y_submission[i] > cutoff:
            y_pred.append(1)
        else:
            y_pred.append(0)
    target_names = ['class 0', 'class 1']
    print "Cutoff = {}".format(cutoff)
    print(classification_report(y_suhas, y_pred, target_names=target_names))
    print confusion_matrix(y_suhas, y_pred)
    print("*********************************************************")

Cutoff = 0.0
             precision    recall  f1-score   support

    class 0       0.00      0.00      0.00     25566
    class 1       0.02      1.00      0.03       439

avg / total       0.00      0.02      0.00     26005

[[    0 25566]
 [    0   439]]
*********************************************************
Cutoff = 0.01
             precision    recall  f1-score   support

    class 0       0.00      0.00      0.00     25566
    class 1       0.02      1.00      0.03       439

avg / total       0.00      0.02      0.00     26005

[[    0 25566]
 [    0   439]]
*********************************************************
Cutoff = 0.02
             precision    recall  f1-score   support

    class 0       0.00      0.00      0.00     25566
    class 1       0.02      1.00      0.03       439

avg / total       0.00      0.02      0.00     26005

[[    0 25566]
 [    0   439]]
*********************************************************
Cutoff = 0.03
             precision    recall 

In [157]:
len(y_submission)

86681

In [165]:
cutoff = 0.76
y_pred = list()
for i in range(0,len(y_submission)):
    if y_submission[i] > cutoff:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [166]:
print len(y_pred)
print sum(y_pred)

86681
4161


In [167]:
# Import the train dataset
data_test = pd.read_csv('test.csv')

In [168]:
submission = pd.DataFrame({'user_id': data_test['user_id'], 'prediction(adopter)':y_pred})
submission = submission[['user_id', 'prediction(adopter)']]
submission.to_csv("stacking_result_29th_Nov_0.76_new.csv", index=False)